## [적대적 예제 생성(Adversarial example Generation)](https://tutorials.pytorch.kr/beginner/fgsm_tutorial.html)
> 이 튜토리얼은 ML 모델들의 보안 취약점에 대한 인식을 높이고, 요즘 화두가 되고 있는 적대적 머신러닝에 대한 통찰력을 제공할 것입니다. 이미지에 눈치챌 수 없는 작은 변화(perturbation)를 추가하면 모델 성능이 크게 달라질 수 있다는 사실에 놀랄 수 있습니다. 이번 튜토리얼에서는 이미지 분류기의 예제를 통해 위 내용에 대해 살펴볼 것입니다. 특히 우리가 가장 많이 사용되는 공격 방법 중 하나인 FGSM(Fast Gradient Sign Attack)을 이용해 MNIST 분류기를 속여 볼 것입니다.

In [1]:
from __future__ import print_function

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torchvision import datasets, transforms

import numpy as np 
import matplotlib.pyplot as plt 
plt.ion()


In [2]:
# MNIST 데이터셋을 내려받을 때, "User-agent" 관련한 제한을 푸는 코드입니다.  
# 더 자세한 내용: https://tutorials.pytorch.kr/beginner/fgsm_tutorial.html
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

## 구현
> 이 섹션에서는 튜토리얼의 입력 매개 변수에 대해 설명하고 공격중인 모델을 정의한 다음 공격을 코딩하고 일부 테스트를 실행합니다.  
  
### 입력
- epsilons: 실행에 사용할 엡실론의 리스트입니다. 엡실론 0의 값은 원래 테스트 셋의 모델 성능을 나타내므로 목록에 유지하는 것이 중요합니다. 또한 직관적으로 엡실론이 클수록 작은 변화가 더 눈에 띄지만 모델 정확도를 저하 시키는 측면에서 더 효과가 있습니다. 여기서 데이터의 범위는 0-1 이기 때문에 엡실론의 값은 1을 초과할 수 없습니다. 
- pretrained_model: 미리 학습된 모델
- use_cuda: CUDA를 사용할지 말지 정하는 이진 플래그

In [3]:
epsilons = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
pretrained_model = './pretrained_models/lenet_mnist_model.pth'
use_cuda = False

### 공격을 받는 모델
> 공격을 받는 모델은 pytorch/examples/mnist와 동일한 MNIST 모델입니다. 본인의 NMIST 모델을 학습 및 저장하는 방식으로 하거나 제공된 모델을 다운로드 해 사용하는 식으로 진행할 수 있습니다. 이 섹션의 목적은 모델과 데이터 로더를 정의한 다음, 모델을 초기화하고 미리 학습된 가중치를 읽어오는 것입니다.

In [ ]:
# LeNet model
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    
    nn.ReLU()